In [14]:
import pandas as pd
import numpy as np
np.random.seed(123)
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['font.size']=8
matplotlib.rcParams['figure.figsize']=(5,5)
import seaborn as sns
from IPython.core.pylabtools import figsize
import scipy
from scipy.stats import percentileofscore
from scipy import stats
import math
from sklearn import datasets, linear_model
from mpl_toolkits.mplot3d import Axes3D

In [2]:
file_name='newedit.csv'
df_data=pd.read_csv(file_name,sep=',',index_col=None, encoding='ISO-8859-1')

In [4]:
print('The size of the dataset: ', df_data.shape)

The size of the dataset:  (67525, 16)


In [5]:
df_data.head(2).append(df_data.tail(2))

C:\Users\TracyCui\AppData\Local\Temp\ipykernel_9644\768104290.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data.head(2).append(df_data.tail(2))


,idex,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count
0,1,google,none,11-Dec-18,Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5,4,5,5,4,5,0
1,2,google,"Mountain View, CA",21-Jun-13,Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4,2,3,3,5,3,2094
67523,67528,microsoft,none,15-Dec-10,Current Employee - Senior Test Lead,"It's a competitive work place, with overload w...","Smart people around you, can learn from them","Politics, weak moral, leaning loyalty",none,3,2,none,3,3.5,3,0
67524,67529,microsoft,none,14-Dec-10,Former Employee - Senior Director,Used to be great,"Compensation, Health benefits and brand name r...",Leadership was better in the late 90's....its ...,Good people keep leaving...you have to ask you...,2,2,none,2,4,2,0


In [6]:
print('Dataset Information: ')
df_data.info()

Dataset Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67525 entries, 0 to 67524
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   idex                        67525 non-null  int64 
 1   company                     67525 non-null  object
 2   location                    67525 non-null  object
 3   dates                       67525 non-null  object
 4   job-title                   67525 non-null  object
 5   summary                     67405 non-null  object
 6   pros                        67525 non-null  object
 7   cons                        67525 non-null  object
 8   advice-to-mgmt              67228 non-null  object
 9   overall-ratings             67525 non-null  int64 
 10  work-balance-stars          67525 non-null  object
 11  culture-values-stars        67525 non-null  object
 12  carrer-opportunities-stars  67525 non-null  object
 13  comp-benefit-stars      

In [7]:
df_data.rename(columns={'overall-ratings':'overall'},inplace=True)
df_data.rename(columns={'work-balance-stars':'rating1'},inplace=True)
df_data.rename(columns={'culture-values-stars':'rating2'},inplace=True)
df_data.rename(columns={'carrer-opportunities-stars':'rating3'},inplace=True)
df_data.rename(columns={'comp-benefit-stars':'rating4'},inplace=True)
df_data.rename(columns={'senior-mangemnet-stars':'rating5'},inplace=True)

In [8]:
df_data = df_data.applymap(lambda x: x if x!='none' else np.nan)
df_data = df_data.dropna(subset=['rating1','rating2','rating3','rating4','rating5'])

In [10]:
dtypes={'rating1':'int64',
        'rating2':'int64',
        'rating3':'int64',
        'rating4':'int64',
        'rating5':'int64',
        'company':'string',
        'location':'string',
        'job-title':'string',
        'summary':'string',
        'pros':'string',
        'cons':'string',
        'advice-to-mgmt':'string'}
df_data=df_data.astype(dtypes)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53218 entries, 0 to 66981
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idex            53218 non-null  int64 
 1   company         53218 non-null  string
 2   location        35711 non-null  string
 3   dates           53218 non-null  object
 4   job-title       53218 non-null  string
 5   summary         53135 non-null  string
 6   pros            53218 non-null  string
 7   cons            53217 non-null  string
 8   advice-to-mgmt  30371 non-null  string
 9   overall         53218 non-null  int64 
 10  rating1         53218 non-null  int64 
 11  rating2         53218 non-null  int64 
 12  rating3         53218 non-null  int64 
 13  rating4         53218 non-null  int64 
 14  rating5         53218 non-null  int64 
 15  helpful-count   53218 non-null  int64 
dtypes: int64(8), object(1), string(7)
memory usage: 6.9+ MB


In [13]:
df_data.isnull().sum()

idex                  0
company               0
location          17507
dates                 0
job-title             0
summary              83
pros                  0
cons                  1
advice-to-mgmt    22847
overall               0
rating1               0
rating2               0
rating3               0
rating4               0
rating5               0
helpful-count         0
dtype: int64

In [15]:
from sklearn import linear_model

x = df_data[['rating1','rating2','rating3','rating4','rating5']].values
y = df_data['overall'].values

regr = linear_model.LinearRegression()
regr.fit(x,y)

print('Coefficients:', regr.coef_)
print('Intercept:', regr.intercept_)

Coefficients: [0.12983305 0.31597821 0.22769739 0.11095756 0.16580561]
Intercept: 0.38571286234751323


In [21]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.25,
                                                    random_state=123)
#对训练集拟合模型
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
print('Coefficients:', regr.coef_)
print('Intercept:', regr.intercept_)

Coefficients: [0.13047211 0.31588424 0.22815057 0.10916584 0.16551276]
Intercept: 0.38949301576008954


In [22]:
from sklearn import metrics
y_pred = regr.predict(X_test)

print('Test MAE:',metrics.mean_absolute_error(y_test, y_pred))
print('Test MSE:',metrics.mean_squared_error(y_test, y_pred))

Test MAE: 0.46645609898630536
Test MSE: 0.3908872801723313
